In [1]:
import numpy as np

from grg_pheno_sim.multi_grg_phenotype import sim_phenotypes_multi_grg
from grg_pheno_sim.model import grg_causal_mutation_model


The following commands only serve the purpose of converting the VCF zip file into GRGs that will be used for phenotype simulation.

In [2]:
%%script bash --out /dev/null
if [ ! -f test-200-samples.grg ]; then
  grg construct -p 10 -t 2 ../data/test-200-samples.vcf.gz --out-file test-200-samples.grg
fi

In [3]:
%%script bash --out /dev/null
if [ ! -f test-200-samples_copy.grg ]; then
  grg construct -p 10 -t 2 ../data/test-200-samples.vcf.gz --out-file test-200-samples_copy.grg
fi

In [4]:
grg_list = ["test-200-samples.grg", "test-200-samples_copy.grg"]
#this is the list of GRG files to be loaded in 

We will first demonstrate loading all GRG files into RAM and simulating phenotypes. Causal mutations are sampled from each GRG, and the genetic values are obtained for the samples. The combined genetic dataframe is the addition of each GRG's genetic values (for each causal mutation ID). Noise is sampled at the end and added to obtain the phenotypes.

NOTE: It is necessary for each GRG to have the same number of samples.

In [5]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_phenotype = True #check for normalizing phenotyopes

normalize_genetic_values_before_noise = False

noise_heritability = [0.33, 0.25]

load_all_into_RAM = True #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, normalize_phenotype=normalize_phenotype, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, noise_heritability=noise_heritability)


Loaded test-200-samples.grg into RAM
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      -5.134548                   0
1                0      -9.021024                   1
2                1       6.558765                   0
3                1     -17.808243                   1
4                2       7.304045                   0
..             ...            ...                 ...
395            197      13.735551                   1
396            198       8.157136                   0
397            198      -5.265874                   1
398            199       5.862298                   0
399            199       2.163684                   1

[400 rows x 3 columns]
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      26.690956                   0
1                0      

In [6]:
multi_grg_multi_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-0.239503,0.591204,0.351701
1,1,0,0.107259,-0.370100,-0.262842
2,0,1,-0.729601,0.227232,-0.502369
3,1,1,0.043305,0.440974,0.484279
4,0,2,-0.644610,0.275756,-0.368854
...,...,...,...,...,...
395,1,197,0.513852,-1.524345,-1.010493
396,0,198,0.303885,1.030478,1.334363
397,1,198,0.617959,1.359482,1.977441
398,0,199,0.716128,0.570329,1.286457


We now perform similar simulations, but by loading the GRGs into RAM sequentially (instead of all together).

In [7]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_genetic_values_before_noise = False

noise_heritability = [0.33, 0.25]

load_all_into_RAM = False #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_seq_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, noise_heritability=noise_heritability)


Loaded test-200-samples.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      11.182723                   0
1                0      -4.319601                   1
2                1      34.405176                   0
3                1      -2.440338                   1
4                2      23.234499                   0
..             ...            ...                 ...
395            197       8.606593                   1
396            198      19.161295                   0
397            198       9.832918                   1
398            199       5.208687                   0
399            199      -4.313657                   1

[400 rows x 3 columns]
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0     -16.079313                   0
1                0      

In [8]:
multi_grg_multi_seq_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-4.896590,-6.331555,-11.228145
1,1,0,10.417591,-12.623364,-2.205773
2,0,1,19.104011,-10.798343,8.305668
3,1,1,3.151106,15.877467,19.028574
4,0,2,4.443349,-4.565012,-0.121663
...,...,...,...,...,...
395,1,197,2.061537,34.773316,36.834853
396,0,198,0.958247,-3.659981,-2.701733
397,1,198,10.558254,33.403477,43.961731
398,0,199,-16.444527,19.764664,3.320137
